In [295]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [296]:
from rBergomi import rBergomi
import numpy as np
import matplotlib.pyplot as plt

In [298]:
model = rBergomi(nsteps = 52, npaths = 10, T = 1.0, alpha = -0.43, eta=0, rho=0)

In [299]:
Y = model.sim_Volterra()

In [300]:
V = model.sim_V()

In [497]:
from itertools import product
import scipy.special

def WZ(n, H, rho):
    idx = np.linspace(1,n,n)
    v, u = np.zeros(pow(n,2)), np.zeros(pow(n,2))

    k = 0
    for i in product(idx, idx):
        v[k], u[k] = i
        k = k + 1

    D = np.sqrt(2.0*H)/(H+0.5)
    res = rho * D * (v**(H+0.5)-(v-np.minimum(u,v))**(H+0.5))
    return np.reshape(res, (n,n))

def G(x, H):
    gamma = 0.5 - H
    F = scipy.special.hyp2f1(1.0,gamma,2.0-gamma,1.0/x)
    return (1.0-2.0*gamma)/(1.0-gamma)*x**(-gamma)*F

def WW(n, H):
    cov = np.zeros((n,n))

    for i in range(n):
        v = i + 1
        for j in range(n):
            u = j + 1
            if i == j:
                cov[i,j] = u**(2.0*H)
            if i > j:
                x = v/u
                cov[i,j] = u**(2.0*H)*G(x,H)
            if i < j:
                cov[i,j] = 0

    return cov + cov.T - np.diag(cov.diagonal())

def ZZ(n):
    cov = np.zeros((n,n))

    for i in range(n):
        v = i + 1
        for j in range(n):
            u = j + 1
            cov[i,j] = np.min([v,u])
    return cov

def check_symmetric(a):
    return np.allclose(a, a.T)

def joint_cov(n,H,rho):
    res = np.block([[WW(n,H), WZ(n,H,rho)],[WZ(n,H,rho).T, ZZ(n)]])
    if check_symmetric(res):
        return res
    else:
        print('Covariance matrix is not symmetric...')
    

def rB_jcov(n, H, rho):
    normal = np.random.normal(size=n*2)
    cov = joint_cov(n,H,rho)
    sigma = np.transpose(scipy.linalg.cholesky(cov))
    control = np.all(np.matmul(sigma,np.transpose(sigma)) - cov < 1e-06)
    if control:
        print("Good news, everyone! I think I perfected a scheme that will simulate all paths!")
        return sigma.dot(normal)
    else: 
        print("Bad news, everyone! I don't think the simulation is going to make it...")

In [498]:
n = 100; H = 0.1; rho = -0.1
rB_jcov(n,H,rho)

Good news, everyone! I think I perfected a scheme that will simulate all paths!


array([-2.68306954e-01,  1.64928002e-01, -4.60627503e-01, -6.81737072e-01,
        4.14969562e-02, -4.61661811e-01, -1.58924792e+00, -2.58329691e+00,
       -1.22579676e+00, -6.10581445e-01, -2.87235430e+00, -1.85996700e+00,
       -9.55135701e-01, -1.80330341e-01, -1.51664354e+00, -1.24242853e+00,
       -1.23548349e+00, -1.30954895e+00, -1.61456910e+00, -1.90494223e+00,
        1.17162877e+00, -1.29256316e+00,  4.03437169e-01, -7.76936464e-01,
       -1.90752732e+00, -7.53871088e-01, -2.84205391e-01, -1.95738780e+00,
       -7.99260309e-01, -8.79636196e-01, -2.53572865e+00, -1.07123079e+00,
       -1.65338036e+00, -1.21828495e+00, -3.11116998e-01, -1.10981387e+00,
        6.25060483e-01, -1.24412866e+00,  1.20562132e+00, -9.66423524e-01,
       -1.50716985e+00, -2.24802901e+00, -1.29048873e+00, -1.08238418e+00,
       -1.33958005e+00, -1.38573377e+00, -5.98132920e-01,  3.38305056e-01,
       -7.17840519e-01,  1.45508052e+00, -1.04223784e+00, -1.06300992e+00,
       -1.70481853e-01, -